In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBRegressor ,XGBClassifier
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
import xgboost as xgb
import optuna
import tqdm
import warnings

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-aug-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-aug-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')

In [ ]:
train = train.drop( columns = 'id')
train

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = StandardScaler()
mm = MinMaxScaler()

In [ ]:
#train , train_extra = train_test_split(train , test_size = 0.9)

pred_col = 'loss'

y_train = train[pred_col]
x_train = train.drop(columns = pred_col)


features = [f'f{i}' for i in range(100)]
x_train = mm.fit_transform(x_train[features])
test = mm.transform(test[features])

x_train

In [ ]:

def xgboost_objective(trial,data=x_train,target=y_train):
    
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25,random_state=42)
    params = {
        "n_estimators": trial.suggest_int("n_estimators",200,2000,100),
        "subsample": trial.suggest_discrete_uniform("subsample",0.6,1,0.1),
        "colsample_bytree": trial.suggest_discrete_uniform("colsample_bytree",0.6,1,0.1),
        "eta": trial.suggest_loguniform("eta",1e-3,0.1),
        "reg_alpha": trial.suggest_int("reg_alpha",1,50),
        "reg_lambda": trial.suggest_int("reg_lambda",5,100),
        "max_depth": trial.suggest_int("max_depth",5,20),
        "min_child_weight": trial.suggest_int("min_child_weight",5,20),
    }
    model = xgb.XGBRegressor(**params, tree_method='gpu_hist', random_state=42)
    model.fit(X_train,y_train,eval_set=[(X_test,y_test)],verbose = False,eval_metric='rmse')

    y_preds = model.predict(X_test)
    loss = np.sqrt(mean_squared_error(y_test, y_preds))
    
    return loss

study = optuna.create_study(direction='minimize')
study.optimize(xgboost_objective, n_trials=25)
print('Number of finished trials:', len(study.trials))
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))


In [ ]:
display(optuna.visualization.plot_parallel_coordinate(study))
#display(optuna.visualization.plot_optimization_history(study))
display(optuna.visualization.plot_slice(study))


In [ ]:
xgb_params = study.best_trial.params
xgb_params['tree_method'] = 'gpu_hist'
xgb_params['random_state'] = 42
test_preds=None

xgb_params

In [ ]:
model_xgboost = XGBRegressor( **xgb_params) # define

model_xgboost.fit(x_train, y_train) 

preds = model_xgboost.predict(test)
print(preds.shape)
preds

plt.bar(range(len(model_xgboost.feature_importances_)), model_xgboost.feature_importances_ )
plt.show()
df_sub = pd.read_csv('../input/tabular-playground-series-aug-2021/sample_submission.csv')
df_sub['loss'] = preds
df_sub.to_csv('submission.csv' , index = False)
df_sub.head(10)
